## Capstone 1 - Part 2

In the first part of the capstone, we focused on Data Retrieval, Data Preprocessing, Feature Engineering and Exploratory Data Analysis using Python & its libraries. Now we are going to shift gears and gain insights into our HR Analytics dataset using SQL.

## TODO: Make use of SQL to do the following:

### Create a SQLITE3 DB using the CSV file (2 pts). Please refer this [link](https://www.linkedin.com/pulse/accessing-sqlite3-database-from-jupyter-notebook-using-varun-lobo/) and this [link](https://www.geeksforgeeks.org/how-to-import-csv-file-in-sqlite-database-using-python/) to know more.

### Calculate the Attrition Rate and summarize attrition (3 pts) by:
- Gender
- Department
- Age
- Average monthly income by job level
- Years at company

### Continue using SQL to explore main reasons for attrition (3 pts), For example:
- Why do more people over 50 years old leave the company than people who aged 40-50?
- Why do people with higher pay still leave the company?
- Which factors drive employees who work at company less than 5 years to leave?

### Effective Communication (2 pts)
- Please make use of markdown cells to communicate your thought process, why did you think of performing a step? what was the observation from the query? etc.
- The code should be commented so that it is readable for the reviewer.

### Grading and Important Instructions
- Each of the above steps are mandatory and should be completed in good faith
- Make sure before submitting that the code is in fully working condition
- It is fine to make use of ChatGPT, stackoverflow type resources, just provide the reference links from where you got it
- Debugging is an art, if you find yourself stuck with errors, take help of stackoverflow and ChatGPT to resolve the issue and if it's still unresolved, reach out to me for help.
- You need to score atleast 7/10 to pass the project, anything less than that will be marked required, needing resubmission.
- Feedback will be provided on 3 levels (Awesome, Suggestion, & Required). Required changes are mandatory to be made.
- For submission, please upload the project on github and share the link to the file with us through LMS.

#### Calculate Attrition Rate


In [2]:
### Implement code

import sqlite3
import pandas as pd

# Load CSV file into a DataFrame
df = pd.read_csv("/Users/vijeethvj8/Downloads/Elevateme/Caspstone 1/HR-Analytics.csv")  

# Create SQLite3 database and establish connection
conn = sqlite3.connect("HR_Analytics.db")
cursor = conn.cursor()

# Store DataFrame into SQLite
# Convert column names to valid SQL column names
df.to_sql("hr_data", conn, if_exists="replace", index=False)

# Query to check if data is loaded successfully
query = "SELECT * FROM hr_data LIMIT 5;"
pd.read_sql(query, conn)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [3]:
# Calculate Attrition Rate
total_employees = pd.read_sql("SELECT COUNT(*) as total FROM hr_data", conn).iloc[0,0]
attrition_count = pd.read_sql("SELECT COUNT(*) as attrition FROM hr_data WHERE Attrition='Yes'", conn).iloc[0,0]
attrition_rate = (attrition_count / total_employees) * 100

# Format output similar to the provided sample
attrition_summary_query = """
SELECT 
    CASE WHEN Attrition='Yes' THEN 'true' ELSE 'false' END AS Attrition, 
    ROUND((COUNT(*) * 100.0 / (SELECT COUNT(*) FROM hr_data)), 1) || '%' AS Attrition_rate
FROM hr_data
GROUP BY Attrition;
"""
print("Attrition Summary:")
print(pd.read_sql(attrition_summary_query, conn))

Attrition Summary:
  Attrition Attrition_rate
0     false          83.9%
1      true          16.1%


#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857886?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4NiwiaG1hYyI6ImUyNGU0YWRhNWQwMDkxNWE2NWNiY2EwNGFlNDNiMTBlYmYyNWY0YzAiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Gender

In [4]:
### Implement code

# Summarize attrition by Gender
attrition_by_gender_query = """
SELECT 
    CASE WHEN Attrition='Yes' THEN 'true' ELSE 'false' END AS Attrition,
    Gender, 
    COUNT(*) AS Count_gender, 
    ROUND((COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(PARTITION BY Gender)), 1) AS Attrition_by_gender
FROM hr_data
GROUP BY Attrition, Gender;
"""
print("Attrition by Gender:")
print(pd.read_sql(attrition_by_gender_query, conn))

Attrition by Gender:
  Attrition  Gender  Count_gender  Attrition_by_gender
0     false  Female           501                 85.2
1      true  Female            87                 14.8
2     false    Male           732                 83.0
3      true    Male           150                 17.0


#### Sample output

![image](https://api-v4.skyprepapp.com/public_api/da/857882?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4MiwiaG1hYyI6IjM1NDU5N2E4MjZmMTMxNWFhMTA4NzFiYTFkZjQ5YjBhNTYyOWYwMzgiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Dept

In [5]:
#### Find Attrition by Gender### Implement code

attrition_by_department_query = """
SELECT Department, Attrition, COUNT(*) AS Department_attrition 
FROM hr_data
GROUP BY Department, Attrition;
"""
print("Attrition by Department:")
print(pd.read_sql(attrition_by_department_query, conn))

Attrition by Department:
               Department Attrition  Department_attrition
0         Human Resources        No                    51
1         Human Resources       Yes                    12
2  Research & Development        No                   828
3  Research & Development       Yes                   133
4                   Sales        No                   354
5                   Sales       Yes                    92


#### Sample Output

<img src="https://api-v4.skyprepapp.com/public_api/da/857884?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4NCwiaG1hYyI6IjdmOTA4NzJjODZmYTgxOGUyMzkyYTBlZjhjYjljNjA4ZGM3NjkzMWYiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9" width="450" height="450">

#### Find Attrition by Age Groups

In [9]:
# Summarize attrition by Age Group (Fixed Grouping Issue)
attrition_by_age_query = """
SELECT 
    CASE 
        WHEN Age < 30 THEN 'Under 30'
        WHEN Age BETWEEN 30 AND 39 THEN '30 - 40'
        WHEN Age BETWEEN 40 AND 49 THEN '40 - 50'
        ELSE 'Over 50' 
    END AS Age_group,
    Attrition,
    COUNT(*) AS num,
    ROUND((COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(PARTITION BY 
        CASE 
            WHEN Age < 30 THEN 'Under 30'
            WHEN Age BETWEEN 30 AND 39 THEN '30 - 40'
            WHEN Age BETWEEN 40 AND 49 THEN '40 - 50'
            ELSE 'Over 50' 
        END
    )), 2) AS percent_by_age
FROM hr_data
GROUP BY 
    CASE 
        WHEN Age < 30 THEN 'Under 30'
        WHEN Age BETWEEN 30 AND 39 THEN '30 - 40'
        WHEN Age BETWEEN 40 AND 49 THEN '40 - 50'
        ELSE 'Over 50' 
    END, Attrition;
"""
print("Attrition by Age Group:")
print(pd.read_sql(attrition_by_age_query, conn))

Attrition by Age Group:
  Age_group Attrition  num  percent_by_age
0   30 - 40        No  533           85.69
1   30 - 40       Yes   89           14.31
2   40 - 50        No  315           90.26
3   40 - 50       Yes   34            9.74
4   Over 50        No  150           86.71
5   Over 50       Yes   23           13.29
6  Under 30        No  235           72.09
7  Under 30       Yes   91           27.91


#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857885?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4NSwiaG1hYyI6IjcwNGVlOWIwYzg1MmMwNmNhODg0NmYyZDFlNWE3OTU1MTFhMGVmYWYiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Monthly Income

In [7]:
### Implement code

# Summarize attrition by Monthly Income
attrition_by_income_query = """
SELECT Department, JobLevel, 
    ROUND(AVG(MonthlyIncome), 1) AS avg_income,
    ROUND(AVG(CASE WHEN Attrition='Yes' THEN MonthlyIncome ELSE NULL END), 1) AS attrition_avg_income,
    ROUND(AVG(CASE WHEN Attrition='Yes' THEN MonthlyIncome ELSE NULL END) - AVG(MonthlyIncome), 1) AS difference
FROM hr_data
GROUP BY Department, JobLevel;
"""
print("Attrition by Job Level and Income:")
print(pd.read_sql(attrition_by_income_query, conn))

Attrition by Job Level and Income:
                Department  JobLevel  avg_income  attrition_avg_income  \
0          Human Resources         1      2733.2                2415.7   
1          Human Resources         2      5563.5                   NaN   
2          Human Resources         3      9623.0               10216.0   
3          Human Resources         4     16147.5                   NaN   
4          Human Resources         5     19197.9                   NaN   
5   Research & Development         1      2840.1                2687.4   
6   Research & Development         2      5291.2                5372.0   
7   Research & Development         3     10170.5                9503.8   
8   Research & Development         4     15634.7               12169.0   
9   Research & Development         5     19218.5               19550.0   
10                   Sales         1      2506.7                2373.4   
11                   Sales         2      5746.1                5917.0   
12 

#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857883?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4MywiaG1hYyI6Ijg3NTU4ZDU1ZjRjN2U1YWI3ODQzYjM1NzFkNjBjMjEwNGY5NWI5ODUiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Years At Company

In [8]:
### Implement code

# Summarize attrition by Tenure (Years at Company)
attrition_by_tenure_query = """
SELECT 
    CASE 
        WHEN YearsAtCompany = 0 THEN 'New Hires'
        WHEN YearsAtCompany BETWEEN 2 AND 5 THEN '2-5 years'
        WHEN YearsAtCompany BETWEEN 6 AND 10 THEN '6-10 years'
        WHEN YearsAtCompany BETWEEN 11 AND 20 THEN '11-20 years'
        ELSE 'Over 20 years' 
    END AS tenure_years,
    COUNT(*) AS num,
    ROUND((COUNT(*) * 100.0 / (SELECT COUNT(*) FROM hr_data WHERE Attrition='Yes')), 1) AS percent
FROM hr_data
WHERE Attrition='Yes'
GROUP BY tenure_years;
"""
print("Attrition by Tenure:")
print(pd.read_sql(attrition_by_tenure_query, conn))

Attrition by Tenure:
    tenure_years  num  percent
0    11-20 years   12      5.1
1      2-5 years   87     36.7
2     6-10 years   55     23.2
3      New Hires   16      6.8
4  Over 20 years   67     28.3


#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857881?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4MSwiaG1hYyI6IjFhYmY3NGI4MzQ1NzViMWZkNDJlMjcwYTUyOTQ0OWQwZjJjMjhmNWUiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Continue using SQL to explore main reasons for attrition (3 pts), For example:
- Why do more people over 50 years old leave the company than people who aged 40-50?
- Why do people with higher pay still leave the company?
- Which factors drive employees who work at company less than 5 years to leave?

In [10]:
# 1. Why do more people over 50 years old leave the company than people aged 40-50?
attrition_by_senior_age_query = """
SELECT 
    CASE 
        WHEN Age >= 50 THEN 'Over 50'
        WHEN Age BETWEEN 40 AND 49 THEN '40-50'
    END AS Age_group,
    COUNT(*) AS num_leaving,
    ROUND(AVG(JobSatisfaction), 1) AS avg_satisfaction,
    ROUND(AVG(WorkLifeBalance), 1) AS avg_worklife_balance,
    ROUND(AVG(JobInvolvement), 1) AS avg_involvement
FROM hr_data
WHERE Attrition = 'Yes' AND Age >= 40
GROUP BY Age_group;
"""
print("Attrition Comparison: Over 50 vs. 40-50")
print(pd.read_sql(attrition_by_senior_age_query, conn))


Attrition Comparison: Over 50 vs. 40-50
  Age_group  num_leaving  avg_satisfaction  avg_worklife_balance  \
0     40-50           34               2.4                   2.6   
1   Over 50           23               2.8                   2.6   

   avg_involvement  
0              2.6  
1              2.6  


In [11]:
# 2. Why do people with higher pay still leave the company?
attrition_by_income_query = """
SELECT 
    JobLevel, 
    ROUND(AVG(MonthlyIncome), 1) AS avg_income,
    COUNT(*) AS num_leaving,
    ROUND(AVG(JobSatisfaction), 1) AS avg_satisfaction,
    ROUND(AVG(WorkLifeBalance), 1) AS avg_worklife_balance
FROM hr_data
WHERE Attrition = 'Yes'
GROUP BY JobLevel
ORDER BY JobLevel;
"""
print("Attrition by Income Level")
print(pd.read_sql(attrition_by_income_query, conn))

Attrition by Income Level
   JobLevel  avg_income  num_leaving  avg_satisfaction  avg_worklife_balance
0         1      2598.1          143               2.4                   2.7
1         2      5759.8           52               2.5                   2.6
2         3      9388.4           32               2.6                   2.7
3         4     13150.4            5               2.6                   2.0
4         5     19463.8            5               2.4                   3.0


In [12]:
# 3. Which factors drive employees who work at the company less than 5 years to leave?
attrition_by_tenure_factors_query = """
SELECT 
    YearsAtCompany, 
    COUNT(*) AS num_leaving,
    ROUND(AVG(JobSatisfaction), 1) AS avg_satisfaction,
    ROUND(AVG(EnvironmentSatisfaction), 1) AS avg_env_satisfaction,
    ROUND(AVG(WorkLifeBalance), 1) AS avg_worklife_balance,
    ROUND(AVG(DistanceFromHome), 1) AS avg_distance_from_home
FROM hr_data
WHERE Attrition = 'Yes' AND YearsAtCompany < 5
GROUP BY YearsAtCompany;
"""
print("Attrition Factors for Employees with <5 Years Tenure")
print(pd.read_sql(attrition_by_tenure_factors_query, conn))

Attrition Factors for Employees with <5 Years Tenure
   YearsAtCompany  num_leaving  avg_satisfaction  avg_env_satisfaction  \
0               0           16               2.3                   2.7   
1               1           59               2.5                   2.5   
2               2           27               2.6                   2.3   
3               3           20               2.0                   2.0   
4               4           19               2.7                   2.3   

   avg_worklife_balance  avg_distance_from_home  
0                   2.8                     5.9  
1                   2.7                    12.4  
2                   2.5                    10.6  
3                   3.0                     7.5  
4                   2.4                    12.9  
